In [1]:
import time
import pandas as pd
from selenium import webdriver


# Scrape magicbricks.com for data:

driver = webdriver.Chrome()
driver.get("https://www.magicbricks.com/property-for-sale/residential-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment&cityName=Nagpur")
property_list = []
flag = True
time.sleep(20)
div = driver.find_element('xpath','.//div[@class="l-srp__results flex__item"]')
a = div.find_elements('xpath','.//a')

for element in a:
	try:
		if element.get_attribute("text")[26:].replace("\n", "") != '':
			location = element.get_attribute("text")[26:].replace("\n", "")
			price =  element.find_element('xpath', '../..//parent::div[@class="flex relative clearfix m-srp-card__container"]').find_element_by_xpath('.//div[@class="m-srp-card__price"]').text
			area = element.find_element('xpath', '..').find_element_by('xpath', '..').find_element('xpath', './/div[@class="m-srp-card__summary__info"]').text
			print(price + " " + area + " " + location)
			property_list.append({"localilty": location, "area": area, "price": price})
	
		driver.find_element('xpath', './/a[@class="toc"]').click()
			
	except:
		break	
driver.close()



# Put into DataFrame for cleaning:

df = pd.DataFrame(property_list)
df.to_csv("properties.csv")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@class="l-srp__results flex__item"]"}
  (Session info: chrome=123.0.6312.106); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF68C937072+63090]
	(No symbol) [0x00007FF68C8A2CC2]
	(No symbol) [0x00007FF68C73EC65]
	(No symbol) [0x00007FF68C78499D]
	(No symbol) [0x00007FF68C784ADC]
	(No symbol) [0x00007FF68C7C5B37]
	(No symbol) [0x00007FF68C7A701F]
	(No symbol) [0x00007FF68C7C3412]
	(No symbol) [0x00007FF68C7A6D83]
	(No symbol) [0x00007FF68C7783A8]
	(No symbol) [0x00007FF68C779441]
	GetHandleVerifier [0x00007FF68CD325CD+4238285]
	GetHandleVerifier [0x00007FF68CD6F72D+4488493]
	GetHandleVerifier [0x00007FF68CD67A0F+4456463]
	GetHandleVerifier [0x00007FF68CA105B6+953270]
	(No symbol) [0x00007FF68C8AE58F]
	(No symbol) [0x00007FF68C8A9264]
	(No s

In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import os
from bs4 import BeautifulSoup
from selenium import webdriver
import sys
import json


# In[2]:

chrome_options = webdriver.ChromeOptions()
# chromedriver = "C:/Users/Parakh Gupta/Downloads/setups/chromedriver" # path to the chromedriver executable
# chromedriver = os.path.expanduser(chromedriver)
# print('chromedriver path: {}'.format(chromedriver))
# sys.path.append(chromedriver)
driver = webdriver.Chrome(options=chrome_options)



# In[3]:


def get_house_links( driver):
    house_links=[]
    cities=['Gurgaon']
    for city in cities:
        for i in range(1,3):
            driver.get("https://www.magicbricks.com/property-for-sale/residential-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Residential-House,Villa&page="+str(i)+"&cityName="+str(city))
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            get_data(soup)


# In[4]:


def get_data(soup):
    h=[]
    prop=soup.find_all('div',class_='m-srp-card SRCard')
    for i in prop:
        name=""
        id_=""
        description=""
        url=""
        price=""
        priceInWord=""
        cityName=""
        addressLocality=""
        longitude=""
        latitude=""
        numberOfRooms=""
        bathroom=""
        bedroom=""
        floorSize=""
        floorno=""
        furnshingstatus=""
        agentName=""
        agentCompanyName=""
        agentMaskedmobilenumber=""
        meta=i.find_all('meta')
        for m in meta:
            if m['itemprop']=='name':
                name=m['content']
            if m['itemprop']=='description':
                description=m['content']
            if m['itemprop']=='url':
                url=str('https://www.magicbricks.com'+m['content'])
            if m['itemprop']=='addressLocality':
                addressLocality=m['content']
            if m['itemprop']=='longitude':
                longitude=m['content']
            if m['itemprop']=='latitude':
                latitude=m['content']
            if m['itemprop']=='numberOfRooms':
                numberOfRooms=m['content']
            if m['itemprop']=='floorSize':
                floorSize=m['content']
        s=i.find('span',class_='hidden')
        bathroom=s['data-bathroom']
        bedroom=s['data-bedroom']
        floorno=s['data-floorno']   
        furnshingstatus=s['data-furnshingstatus']
        price=s['data-price']
        id_=s['id']
        ag=soup.find('span',id=id_)
        priceInWord=ag['data-priced']
        cityName=ag['data-cityname']
        agentName=ag['data-soname']
        agentCompanyName=ag['data-companyname']
        agentMaskedmobilenumber=ag['data-maskedmobilenumber']
        var = {
                "name":name,
                "id":id_,
                "description":description,
                "url":url,
                "price":price,
                "priceInWord":priceInWord,
                "location":
                        {
                            "cityName":cityName,
                            "addressLocality":addressLocality,
                            "longitude":longitude,
                            "latitude":latitude,
                        },
                "flatDetails":
                        {
                            "numberOfRooms":numberOfRooms,
                            "bathroom":bathroom,
                            "bedroom":bedroom,
                            "floorSize":floorSize,
                            "floorno":floorno,
                            "furnshingstatus":furnshingstatus,
                        },
                "agentDetails":
                        {
                            "agentName":agentName,
                            "agentCompanyName":agentCompanyName,
                            "agentMaskedmobilenumber":agentMaskedmobilenumber,
                        }

            }
        #print(var)
        h.append(var)
    with open("data.json",'r+') as f:
        feeds = json.load(f)
        for i in h:
            feeds['property'].append(var)
        f.seek(0)
        json.dump(feeds, f)


# In[5]:


v={
    "property":[]
}
with open("data.json", mode='w', encoding='utf-8') as f:
    json.dump(v,f)


# In[ ]:


get_house_links(driver)


# In[ ]: